<a href="https://colab.research.google.com/github/ngrief/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
df = pd.read_csv(url)

# Display first few rows
df.head()



,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop unnecessary columns
df = df.drop(columns=['EIN', 'NAME'])

# Check unique values per column
df.nunique()


,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [4]:
# Identify columns with too many unique values
columns_to_bin = ['APPLICATION_TYPE', 'CLASSIFICATION']

# Function to replace rare values
def bin_rare_categories(df, column, threshold):
    value_counts = df[column].value_counts()
    rare_categories = value_counts[value_counts < threshold].index
    df[column] = df[column].replace(rare_categories, "Other")

# Apply binning
for col in columns_to_bin:
    bin_rare_categories(df, col, threshold=100)  # Adjust threshold as needed

# Verify the changes
df[columns_to_bin].nunique()


,0
APPLICATION_TYPE,10
CLASSIFICATION,12


In [5]:
# One-hot encode categorical variables
df = pd.get_dummies(df)

# Split into features (X) and target (y)
X = df.drop(columns=['IS_SUCCESSFUL'])
y = df['IS_SUCCESSFUL']


In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the optimized neural network model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),  # More neurons
    Dense(64, activation='tanh'),  # Experimenting with tanh activation
    Dense(32, activation='relu'),  # Added extra hidden layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Add callbacks to prevent overfitting
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss")
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train the optimized model with increased epochs and callbacks
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])


Epoch 1/100
843/858 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7036 - loss: 0.5832

858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7040 - loss: 0.5829 - val_accuracy: 0.7268 - val_loss: 0.5605
Epoch 2/100
848/858 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7289 - loss: 0.5540

858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7289 - loss: 0.5540 - val_accuracy: 0.7292 - val_loss: 0.5562
Epoch 3/100
839/858 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7319 - loss: 0.5466

858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7319 - loss: 0.5467 - val_accuracy: 0.7261 - val_loss: 0.5548
Epoch 4/100
842/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7331 - loss: 0.5453

858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7331 - loss: 0.5453 - val_accuracy: 0.7303 - val_loss: 0.5539
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7312 - loss: 0.5512 - val_accuracy: 0.7201 - val_loss: 0.5543
Epoch 6/100
856/858 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7299 - loss: 0.5506

858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7299 - loss: 0.5506 - val_accuracy: 0.7292 - val_loss: 0.5526
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7348 - loss: 0.5411 - val_accuracy: 0.7300 - val_loss: 0.5576
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7332 - loss: 0.5472 - val_accuracy: 0.7303 - val_loss: 0.5534
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7360 - loss: 0.5435 - val_accuracy: 0.7259 - val_loss: 0.5544
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7293 - loss: 0.5471 - val_accuracy: 0.7249 - val_loss: 0.5527
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7360 - loss: 0.5395 - val_accuracy: 0.7268 - val_loss: 0.5529
Epoch 12/100
840/858 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7358 - loss: 0.5412

858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7357 - loss: 0.5412 - val_accuracy: 0.7287 - val_loss: 0.5521
Epoch 13/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7373 - loss: 0.5392 - val_accuracy: 0.7232 - val_loss: 0.5532
Epoch 14/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7345 - loss: 0.5420 - val_accuracy: 0.7277 - val_loss: 0.5558
Epoch 15/100
846/858 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7402 - loss: 0.5385

858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7401 - loss: 0.5386 - val_accuracy: 0.7281 - val_loss: 0.5518
Epoch 16/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7393 - loss: 0.5374 - val_accuracy: 0.7265 - val_loss: 0.5543
Epoch 17/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7381 - loss: 0.5404 - val_accuracy: 0.7268 - val_loss: 0.5535
Epoch 18/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7354 - loss: 0.5408 - val_accuracy: 0.7299 - val_loss: 0.5543
Epoch 19/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7391 - loss: 0.5348 - val_accuracy: 0.7274 - val_loss: 0.5541
Epoch 20/100
849/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7363 - loss: 0.5379

858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7363 - loss: 0.5380 - val_accuracy: 0.7318 - val_loss: 0.5513
Epoch 21/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7375 - loss: 0.5383 - val_accuracy: 0.7270 - val_loss: 0.5529
Epoch 22/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7360 - loss: 0.5403 - val_accuracy: 0.7261 - val_loss: 0.5539
Epoch 23/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7327 - loss: 0.5394 - val_accuracy: 0.7262 - val_loss: 0.5574
Epoch 24/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7436 - loss: 0.5318 - val_accuracy: 0.7277 - val_loss: 0.5531
Epoch 25/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7410 - loss: 0.5324 - val_accuracy: 0.7271 - val_loss: 0.5544
Epoch 26/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7424 - loss: 0.5338 - val_accuracy: 0.7241 - val_loss: 0.5588
Epoch 27/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7405 - loss: 0.5299 - val_accuracy:

In [10]:
# Evaluate the optimized model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Optimized Test Accuracy: {accuracy:.4f}")


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7308 - loss: 0.5500
Optimized Test Accuracy: 0.7318


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the previous best model
model = Sequential([
    Dense(256, activation='relu', input_dim=X_train.shape[1]),  # Best working setup
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks to prevent overfitting
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss")
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train the model again with slightly more epochs
history = model.fit(X_train, y_train, epochs=175, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

# Save the optimized model
model.save("AlphabetSoupCharity_Optimization.h5")


Epoch 1/175
848/858 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7123 - loss: 0.5781

858/858 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7125 - loss: 0.5779 - val_accuracy: 0.7238 - val_loss: 0.5619
Epoch 2/175
846/858 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7260 - loss: 0.5551

858/858 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7260 - loss: 0.5551 - val_accuracy: 0.7296 - val_loss: 0.5597
Epoch 3/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7340 - loss: 0.5473 - val_accuracy: 0.7267 - val_loss: 0.5613
Epoch 4/175
841/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7332 - loss: 0.5470

858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7332 - loss: 0.5471 - val_accuracy: 0.7274 - val_loss: 0.5537
Epoch 5/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7331 - loss: 0.5465 - val_accuracy: 0.7318 - val_loss: 0.5543
Epoch 6/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7349 - loss: 0.5450 - val_accuracy: 0.7274 - val_loss: 0.5557
Epoch 7/175
848/858 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7358 - loss: 0.5425

858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7358 - loss: 0.5425 - val_accuracy: 0.7306 - val_loss: 0.5527
Epoch 8/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7290 - loss: 0.5452 - val_accuracy: 0.7308 - val_loss: 0.5540
Epoch 9/175
854/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7359 - loss: 0.5425

858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7359 - loss: 0.5425 - val_accuracy: 0.7278 - val_loss: 0.5518
Epoch 10/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7354 - loss: 0.5423 - val_accuracy: 0.7296 - val_loss: 0.5518
Epoch 11/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7395 - loss: 0.5390 - val_accuracy: 0.7283 - val_loss: 0.5535
Epoch 12/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7363 - loss: 0.5380 - val_accuracy: 0.7229 - val_loss: 0.5537
Epoch 13/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7393 - loss: 0.5373 - val_accuracy: 0.7324 - val_loss: 0.5536
Epoch 14/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7384 - loss: 0.5399 - val_accuracy: 0.7265 - val_loss: 0.5529
Epoch 15/175
848/858 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7395 - loss: 0.5369

858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7395 - loss: 0.5369 - val_accuracy: 0.7284 - val_loss: 0.5516
Epoch 16/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7343 - loss: 0.5394 - val_accuracy: 0.7300 - val_loss: 0.5522
Epoch 17/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7347 - loss: 0.5369 - val_accuracy: 0.7268 - val_loss: 0.5530
Epoch 18/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7386 - loss: 0.5391 - val_accuracy: 0.7284 - val_loss: 0.5530
Epoch 19/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7393 - loss: 0.5336 - val_accuracy: 0.7232 - val_loss: 0.5531
Epoch 20/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7400 - loss: 0.5345 - val_accuracy: 0.7312 - val_loss: 0.5538
Epoch 21/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7389 - loss: 0.5356 - val_accuracy: 0.7259 - val_loss: 0.5542
Epoch 22/175
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7352 - loss: 0.5358 - val_accuracy:

In [18]:
# Load the best model from training
from tensorflow.keras.models import load_model

# Load the optimized model
model = load_model("AlphabetSoupCharity_Optimization.h5")

# Evaluate the optimized model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Reverted Test Accuracy: {accuracy:.4f}")



215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7280 - loss: 0.5516
Reverted Test Accuracy: 0.7284


In [21]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define optimized batch size and learning rate
batch_size = 64
optimizer = Adam(learning_rate=0.0005)  # Slower learning rate for stability

# Define the optimized neural network model
model = Sequential([
    Dense(256, input_dim=X_train.shape[1]),
    LeakyReLU(alpha=0.1),  # Replaces ReLU with LeakyReLU
    Dropout(0.2),  # Reduces overfitting

    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(64),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model with the adjusted optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks to prevent overfitting
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss")
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train the optimized model
history = model.fit(X_train, y_train, epochs=200, batch_size=batch_size,
                    validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

# Save the optimized model
model.save("AlphabetSoupCharity_Optimization.h5")

# Print completion message
print("Model training complete. Saved as AlphabetSoupCharity_Optimization.h5")


Epoch 1/200
425/429 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6940 - loss: 0.6039

429/429 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6943 - loss: 0.6036 - val_accuracy: 0.7230 - val_loss: 0.5608
Epoch 2/200
421/429 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7316 - loss: 0.5585

429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7315 - loss: 0.5585 - val_accuracy: 0.7246 - val_loss: 0.5603
Epoch 3/200
420/429 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7281 - loss: 0.5562

429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7281 - loss: 0.5562 - val_accuracy: 0.7241 - val_loss: 0.5578
Epoch 4/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7313 - loss: 0.5556 - val_accuracy: 0.7235 - val_loss: 0.5589
Epoch 5/200
418/429 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7310 - loss: 0.5534

429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7310 - loss: 0.5535 - val_accuracy: 0.7290 - val_loss: 0.5536
Epoch 6/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7364 - loss: 0.5449 - val_accuracy: 0.7241 - val_loss: 0.5551
Epoch 7/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7358 - loss: 0.5480 - val_accuracy: 0.7268 - val_loss: 0.5544
Epoch 8/200
421/429 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7260 - loss: 0.5565

429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7261 - loss: 0.5564 - val_accuracy: 0.7258 - val_loss: 0.5533
Epoch 9/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7346 - loss: 0.5506 - val_accuracy: 0.7246 - val_loss: 0.5546
Epoch 10/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7333 - loss: 0.5507 - val_accuracy: 0.7270 - val_loss: 0.5539
Epoch 11/200
423/429 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7342 - loss: 0.5491

429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7342 - loss: 0.5491 - val_accuracy: 0.7292 - val_loss: 0.5519
Epoch 12/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7330 - loss: 0.5501 - val_accuracy: 0.7309 - val_loss: 0.5520
Epoch 13/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7319 - loss: 0.5487 - val_accuracy: 0.7273 - val_loss: 0.5530
Epoch 14/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7298 - loss: 0.5523 - val_accuracy: 0.7310 - val_loss: 0.5529
Epoch 15/200
427/429 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7349 - loss: 0.5442

429/429 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7349 - loss: 0.5442 - val_accuracy: 0.7289 - val_loss: 0.5509
Epoch 16/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7287 - loss: 0.5494 - val_accuracy: 0.7309 - val_loss: 0.5517
Epoch 17/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7290 - loss: 0.5497

429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7290 - loss: 0.5497 - val_accuracy: 0.7309 - val_loss: 0.5505
Epoch 18/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7362 - loss: 0.5468 - val_accuracy: 0.7271 - val_loss: 0.5514
Epoch 19/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7356 - loss: 0.5434 - val_accuracy: 0.7273 - val_loss: 0.5525
Epoch 20/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7343 - loss: 0.5455 - val_accuracy: 0.7293 - val_loss: 0.5519
Epoch 21/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7350 - loss: 0.5402 - val_accuracy: 0.7280 - val_loss: 0.5507
Epoch 22/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7291 - loss: 0.5487 - val_accuracy: 0.7309 - val_loss: 0.5515
Epoch 23/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7345 - loss: 0.5437 - val_accuracy: 0.7280 - val_loss: 0.5529
Epoch 24/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7333 - loss: 0.5467 - val_accuracy:

Model training complete. Saved as AlphabetSoupCharity_Optimization.h5


In [22]:
# Load the best model from training
from tensorflow.keras.models import load_model

# Load the optimized model
model = load_model("AlphabetSoupCharity_Optimization.h5")

# Evaluate the optimized model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Final Optimized Test Accuracy: {accuracy:.4f}")


215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7317 - loss: 0.5480
Final Optimized Test Accuracy: 0.7309
